In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

import wrangle

# Analysis
Anonymize the IP addresses (added function to wrangle.py), then answer the following questions:
1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
    1. PHP: **content/html-css** (262)
    2. Java: **javascript-i/introduction/working-with-data-types-operators-and-variables** (7094)
    3. DS: **classification/overview** (1785)
    4. Front End: **content/html-css**
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
    - **Voyageurs: java-i/introduction-to-java** (447) - 3x the average - 22% higher than next cohort
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
    - **Yes**; PHP and Java students make up nearly all of bottom 100
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
    1. Unauthorized access: **No**, all events have associated user id
    2. Web scraping: **Probably**, there's lots of .html, .json, and .php requests
    3. Suspicious IP addresses: **I'd say no**, all events have associated user id
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
    1. Cross-Access: **Yes**, A user from Teddy (Java) accessed the DS Classification module twice in 2020
    2. Happen Before: **I don't know, the first DS cohort was mid-2019 and the ability changed in 2019**
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
    - Nearly all of top-referenced lessons are in the **Fundamentals module**
7. Which lessons are least accessed?
    - There are a lot of paths that seem to be lessons but only have one request- are they deprecated? I don't feel confident choosing any.
8. Anything else I should be aware of?
    - If you're focused on reducing student reference to the curriculum, let me know, I can look into that further.
    - If you're focused on identifying potential unauthorized access to the curriculum, let me know, I can look into that further.

In [2]:
# import wrangled data
df_original = wrangle.wrangle_cohorts()

In [3]:
df = df_original.copy()
df.head(3)

,date,time,path,user_id,ip,cohort,start_date,end_date,program_id,program_name,subdomain
datetime,,,,,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,PHP Full Stack Web Development,php
2018-01-26 09:56:02,2018-01-26,09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,PHP Full Stack Web Development,php
2018-01-26 09:56:05,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,PHP Full Stack Web Development,php


# Which lesson appears to attract the most traffic consistently across cohorts (per program)?
1. PHP: content/html-css (262)
2. Java: javascript-i/introduction/working-with-data-types-operators-and-variables (7094)
3. DS: classification/overview (1785)
4. Front End: content/html-css

In [4]:
# make path column make sense
df = wrangle.lesson_identification(df) # bool column for identified lessons / non-lessons
df = wrangle.html_php_requests(df) # bool columns for html and php requests
df = wrangle.alter_paths(df) # add column for paths without the .html or .php suffix
df[['path','is_lesson','html_request','php_request','alter_path']].sample(3, random_state=123)

,path,is_lesson,html_request,php_request,alter_path
datetime,,,,,
2020-03-30 16:13:15,/,False,False,False,/
2019-04-15 09:37:17,examples/javascript/defuse-the-bom.html,False,True,False,examples/javascript/defuse-the-bom
2019-10-15 10:42:24,mysql/clauses/order-by,True,False,False,mysql/clauses/order-by


In [5]:
# check top traffic lesson in each program
for program in df.subdomain.unique():
    print(program)
    print(df[df.is_lesson & (df.subdomain == program)].alter_path.value_counts().head(1))

php
content/html-css    262
Name: alter_path, dtype: int64
java
javascript-i/introduction/working-with-data-types-operators-and-variables    7094
Name: alter_path, dtype: int64
fe
content/html-css    2
Name: alter_path, dtype: int64
ds
classification/overview    1785
Name: alter_path, dtype: int64


# Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
- Voyageurs: java-i/introduction-to-java (447) - 3x the average - 22% higher than next cohort

In [6]:
for cohort in df[df.subdomain == 'java'].cohort.unique():
    print(cohort)
    print(df[(df['cohort'] == cohort) & df.is_lesson]\
          .alter_path.value_counts().head(1))

Teddy
mysql/tables    387
Name: alter_path, dtype: int64
Sequoia
spring/fundamentals/views    207
Name: alter_path, dtype: int64
Niagara
spring/setup    13
Name: alter_path, dtype: int64
Pinnacles
javascript-ii/npm    51
Name: alter_path, dtype: int64
Mammoth
spring/fundamentals/controllers    12
Name: alter_path, dtype: int64
Ulysses
spring/fundamentals/form-model-binding    357
Name: alter_path, dtype: int64
Voyageurs
java-i/introduction-to-java    447
Name: alter_path, dtype: int64
Wrangell
html-css/css-ii/bootstrap-grid-system    313
Name: alter_path, dtype: int64
Xanadu
html-css/elements    438
Name: alter_path, dtype: int64
Yosemite
javascript-ii/map-filter-reduce    299
Name: alter_path, dtype: int64
Staff
jquery/ajax/weather-map    1050
Name: alter_path, dtype: int64
Zion
spring/fundamentals/repositories    427
Name: alter_path, dtype: int64
Andromeda
mysql/tables    318
Name: alter_path, dtype: int64
Betelgeuse
html-css/elements    495
Name: alter_path, dtype: int64
Ceres
html

In [7]:
df[df.alter_path == 'java-i/introduction-to-java']\
.groupby('cohort').alter_path.count().sort_values().mean()

# 447 (Voyageur's access count) / 154 (mean) is roughly 3

153.85714285714286

# Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
- Yes; PHP and Java students make up nearly all of bottom 100

In [8]:
low_access = list(df.user_id.value_counts().tail(100).index) # identify lowest 30 active
for user in low_access:
    print(df[df.user_id == user].subdomain.unique())

['php']
['java']
['java']
['php']
['java']
['php']
['java']
['java']
['php']
['java']
['java']
['php']
['php']
['php']
['php']
['java']
['java']
['java']
['java']
['java']
['java']
['php']
['php']
['java']
['java']
['ds']
['ds']
['java']
['php']
['php']
['php']
['ds']
['java']
['php']
['java']
['php']
['php']
['php']
['ds']
['java']
['java']
['java']
['java']
['java']
['php']
['php']
['php']
['ds']
['php']
['java']
['php']
['java']
['java']
['php']
['php']
['ds']
['java']
['php']
['php']
['java']
['php']
['php']
['php']
['java']
['java']
['java']
['java']
['java']
['java']
['php']
['php']
['php']
['fe']
['php']
['java']
['java']
['php']
['php']
['java']
['php']
['php']
['java']
['java']
['java']
['php']
['java']
['java']
['php']
['php']
['java']
['java']
['php']
['java']
['java']
['php']
['php']
['java']
['java']
['java']
['php']


# Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
1. Unauthorized access: No, all events have associated user id
2. Web scraping: Lots of .html, .json, and .php requests, so probably
3. Suspicious IP addresses: All events have associated user id, I'd say no

In [9]:
df.user_id.unique() # all entries have associated user id, so no unauthorized access events logged
# df.groupby('user_id').path.count().sort_values(ascending=False).head(15)

array([  1,  11,  37,  50,  51,  53,  60,  64,  66,  68,  69,  70,  71,
        75,  77,  80,  82,  85,  87,  90,  92,  93,  94,  95,  96,  98,
       101, 102, 105, 106, 109, 112, 113, 114, 139, 140, 141, 144, 146,
       147, 148, 149, 150, 151, 156, 161, 163, 167, 168, 169, 170, 173,
       174, 177,  88, 209, 210, 118, 211, 212, 214, 241, 242, 245, 246,
       248, 257, 279, 280, 281, 283, 311, 312, 313, 314, 316, 318, 348,
       396, 398, 399, 402, 405, 428, 462, 463, 465, 486, 513, 419, 593,
       610, 646, 647, 649, 740, 852, 952,   2,   3,   4,   5,   6,   7,
         8,   9,  10,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,
        22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,
        35,  36,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  49,
        52,  55,  56,  57,  65,  67,  72,  76,  81,  83,  84,  91,  97,
        99, 104, 107, 108, 110, 115, 116, 117, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 13

# At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
1. Yes, A user from Teddy (Java) accessed the DS Classification module twice in 2020
2. I don't know, the first DS cohort was mid-2019 and the ability changed in 2019

In [10]:
# create new dataframe for java cohorts in 2020
df_2020_java = df[(df.index.year == 2020) & (df.subdomain == 'java')]
# check if java cohorts in 2020 accessed the DS classification module
df_2020_java[df_2020_java.path.str.contains('classification')].cohort.value_counts()

Staff    1592
Teddy       2
Name: cohort, dtype: int64

In [11]:
# create a mask to oust the cross-access
mask = df_2020_java.path.str.contains('classification') & (df_2020_java.cohort == 'Teddy')
# check
df_2020_java[mask]

,date,time,path,user_id,ip,cohort,start_date,end_date,program_id,program_name,subdomain,is_lesson,html_request,php_request,alter_path
datetime,,,,,,,,,,,,,,,
2020-11-10 10:14:29,2020-11-10,10:14:29,classification/overview,20,104.190.242.242,Teddy,2018-01-08,2018-05-17,2,Java Full Stack Web Development,java,True,False,False,classification/overview
2020-11-10 10:14:29,2020-11-10,10:14:29,classification/scale_features_or_not.svg,20,104.190.242.242,Teddy,2018-01-08,2018-05-17,2,Java Full Stack Web Development,java,False,False,False,classification/scale_features_or_not.svg


# What topics are grads continuing to reference after graduation and into their jobs (for each program)?
- Nearly all of top-referenced lessons are in the Fundamentals module

In [12]:
# by cohort
for cohort in df.cohort.unique():
    # get end date
    grad_date = df[df.cohort == cohort].end_date.unique()[0]
    # create mask for post-grad lesson access for the cohort
    mask = (df.cohort == cohort) & (df.date > df.end_date) & df.is_lesson
    # add all post-grad path requests to list
    cohort_access_list = df[mask].alter_path.to_list()
    # make a dataframe for all post-grad queries
    grad_df = pd.DataFrame({cohort:cohort_access_list})
    # check most-queried
    highest_row = grad_df.value_counts().head(1)
    # print most-queried (still by cohort)
    print(highest_row)

Hampton          
java-iii/servlets    30
dtype: int64
Arches                   
spring/fundamentals/views    98
dtype: int64
Quincy               
content/laravel/intro    38
dtype: int64
Kings                
content/laravel/intro    83
dtype: int64
Lassen                          
spring/fundamentals/repositories    115
dtype: int64
Glacier         
content/html-css    29
dtype: int64
Denali           
prework/databases    1
dtype: int64
Joshua          
content/html-css    19
dtype: int64
Olympic       
java-i/methods    69
dtype: int64
Badlands                   
content/php_ii/command-line    6
dtype: int64
Ike              
html-css/elements    11
dtype: int64
Franklin         
javascript-ii/es6    10
dtype: int64
Series([], dtype: int64)
Teddy       
spring/setup    87
dtype: int64
Sequoia                  
spring/fundamentals/views    85
dtype: int64
Niagara     
spring/setup    13
dtype: int64
Pinnacles        
javascript-ii/npm    51
dtype: int64
Mammoth                     

# Which lessons are least accessed?
1. There are a lot of paths that seem to be lessons but only have one request- are they deprecated? I don't feel confident choosing any.

In [13]:
# check lowest 50 accessed paths
df[df.is_lesson].groupby('subdomain').alter_path.value_counts().tail(50)

subdomain  alter_path                                            
php        appendix/postwork/trains                                  1
           cohorts/24/capstone-all                                   1
           cohorts/24/grades                                         1
           content/appendix/code-standards/html-css                  1
           content/appendix/javascript/functions/controllers         1
           content/appendix/javascript/functions/models              1
           content/appendix/javascript/functions/scope               1
           content/appendix/javascript/functions/templating          1
           content/appendix/javascript/javascript/functions/scope    1
           content/appendix/postwork/mars-rover                      1
           content/appendix/postwork/sales-tax                       1
           content/appendix/postwork/trains                          1
           content/conditionals                                      1
           

# Anything else I should be aware of?
- If you're focused on reducing student reference to the curriculum, let me know, I can look into that further.
- If you're focused on identifying potential unauthorized access to the curriculum, let me know, I can look into that further.